In [37]:
import pandas as pd
import numpy as np

## Data Cleaning

In [48]:
rawData = pd.read_csv('Evangelist_SqlPerf/data/SqlMetric_prepared.csv')
rawData.head()

FileNotFoundError: [Errno 2] File data/SqlMetric_prepared.csv does not exist: 'data/SqlMetric_prepared.csv'

In [39]:
rawData.drop(['ActivityID', 'Event Name', 'criteria', 'Process Name'], axis=1, inplace=True)

In [40]:
rawData.isnull().sum()

Time MSec             0
DURATION_MSEC    173975
metrics               0
timestamp             0
usn                   0
viewName              0
dtype: int64

In [41]:
rawData.dropna(inplace=True)
print(rawData.isnull().sum())
print('Dataset shape: {}'.format(rawData.shape))

Time MSec        0
DURATION_MSEC    0
metrics          0
timestamp        0
usn              0
viewName         0
dtype: int64
Dataset shape: (173974, 6)


In [42]:
rawData.head()

,Time MSec,DURATION_MSEC,metrics,timestamp,usn,viewName
1,2621.028,102.572,Table_1:47 Table_2:476 Table_3:199 Table_4:56,11/15/2019 9:51,0,Procedure_0
3,2700.351,79.245,Table_1:47 Table_2:476 Table_3:199 Table_4:56,11/15/2019 9:51,0,Procedure_1
5,2701.448,1.073,Table_1:47 Table_2:476 Table_3:199 Table_4:56,11/15/2019 9:51,0,Procedure_2
7,6288.764,"3,587.28",Table_1:47 Table_2:476 Table_3:199 Table_4:56,11/15/2019 9:51,0,Procedure_3
9,6386.315,97.403,Table_1:47 Table_2:476 Table_3:199 Table_4:56,11/15/2019 9:51,"76,986",Procedure_4


In [43]:
rawData['db_state'] = rawData['metrics'].apply(lambda x: x.split(' '))
rawData.drop(['metrics'], axis=1, inplace=True)
print('Dataset shape: {}'.format(rawData.shape))
row = rawData.iloc[0,5]
print('Metrics: {}'.format(row))

dic = {}

for st in row:
    splitted = st.split(':')
    key = splitted[0] 
    dic[key] = [] 
    
print('Parsing sql metrics')
for index, row in rawData.iterrows():
    metrics_row = row['db_state']
    for i in range(0,4):
        st = metrics_row[i]
        splitted = st.split(':')
        key = 'Table_{}'.format(i+1)
        val = np.NaN   
        if len(splitted)>1:
            val = int(splitted[1])
        
        dic[key].append(val)
        

print('Adding new metrics collumns')     
for key in dic.keys():
    rawData[key] = dic[key]

print('Drop db_state column')
rawData.drop(['db_state'], axis=1, inplace=True)
    
print('Parsing sql metrics finnised')
print('Dataset shape: {}'.format(rawData.shape))

Dataset shape: (173974, 6)
Metrics: ['Table_1:47', 'Table_2:476', 'Table_3:199', 'Table_4:56']
Parsing sql metrics
Adding new metrics collumns
Drop db_state column
Parsing sql metrics finnised
Dataset shape: (173974, 9)


In [44]:
print(rawData.isnull().sum())

Time MSec           0
DURATION_MSEC       0
timestamp           0
usn                 0
viewName            0
Table_1          4844
Table_2          4844
Table_3          4844
Table_4          4844
dtype: int64


In [45]:
rawData.dropna(inplace=True)
print('Dataset shape: {}'.format(rawData.shape))

Dataset shape: (169130, 9)


In [46]:
rawData = rawData.applymap(lambda x: x.replace(',','') if type(x) is str else x)
rawData['is_cold_start'] = rawData['usn'].apply(lambda x: True if int(x) == 0 else False)
rawData.drop(['usn'], axis=1, inplace=True)
rawData.rename(columns={'Time MSec': 'timestamp', 'DURATION_MSEC': 'duration', 'timestamp':'time'}, inplace=True)
rawData.head()

,timestamp,duration,time,viewName,Table_1,Table_2,Table_3,Table_4,is_cold_start
1,2621.028,102.572,11/15/2019 9:51,Procedure_0,47.0,476.0,199.0,56.0,True
3,2700.351,79.245,11/15/2019 9:51,Procedure_1,47.0,476.0,199.0,56.0,True
5,2701.448,1.073,11/15/2019 9:51,Procedure_2,47.0,476.0,199.0,56.0,True
7,6288.764,3587.28,11/15/2019 9:51,Procedure_3,47.0,476.0,199.0,56.0,True
9,6386.315,97.403,11/15/2019 9:51,Procedure_4,47.0,476.0,199.0,56.0,False


## Studing data